In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score
from tabulate import tabulate

In [2]:
df = pd.read_csv("rice data.csv")

In [3]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio


In [4]:
df.shape

(75000, 107)

In [5]:
df["CLASS"].value_counts()

CLASS
Basmati      15000
Arborio      15000
Jasmine      15000
Ipsala       15000
Karacadag    15000
Name: count, dtype: int64

In [6]:
#df["CLASS"].replace({"Basmati":0,"Arborio":1,"Jasmine":2,"Ipsala":3,"Karacadag":4},inplace=True)

In [7]:
df["CLASS"].value_counts()

CLASS
Basmati      15000
Arborio      15000
Jasmine      15000
Ipsala       15000
Karacadag    15000
Name: count, dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Columns: 107 entries, AREA to CLASS
dtypes: float64(95), int64(11), object(1)
memory usage: 61.2+ MB


In [9]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4V,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ
count,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,...,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000,75000.000000
mean,8379.197507,378.169453,161.805540,66.829335,0.886077,101.731251,0.975896,8584.862320,0.633226,2.597063,...,0.448960,111.088252,64.379443,61.461457,101.925425,66.240541,63.202088,0.341944,0.357058,0.421176
std,3119.209274,70.597008,36.461005,16.689269,0.071906,17.874070,0.007966,3189.298025,0.123795,0.968982,...,0.021736,5.904854,1.175616,2.435635,5.436861,2.159109,1.174976,0.041921,0.047139,0.043137
min,3929.000000,261.040000,96.968300,34.673000,0.627700,70.728800,0.877500,4032.000000,0.278800,1.284500,...,0.313900,82.300600,59.137900,53.653800,75.191800,58.323800,57.363400,0.159700,0.169000,0.191800
25%,6259.000000,316.431500,132.623500,49.650200,0.846100,89.270400,0.970900,6385.000000,0.561000,1.876100,...,0.434200,106.632900,63.883800,59.465575,97.834400,64.842000,63.052800,0.309900,0.320900,0.391200
50%,7345.000000,351.261000,149.343950,69.183900,0.885600,96.705500,0.976400,7532.000000,0.655800,2.153200,...,0.451600,110.770700,64.419350,61.424400,101.683700,66.291600,63.522050,0.340100,0.353300,0.424200
75%,8901.000000,444.986000,197.462025,75.814125,0.950800,106.457100,0.982200,9153.000000,0.727800,3.228700,...,0.466100,115.065075,65.174200,63.076825,105.592450,68.011800,63.734000,0.370300,0.387900,0.454700
max,21019.000000,593.698000,255.647200,113.441100,0.986800,163.591600,0.992100,21633.000000,0.901700,6.179500,...,0.495100,126.265100,67.459000,70.284000,116.287300,73.424700,66.539100,0.463900,0.488600,0.530200


In [10]:
df = df.fillna(0)

In [11]:
print(sum(df.isnull().sum()))

0


In [12]:
imputer = SimpleImputer(strategy='mean')
df = np.array(df)
df = pd.DataFrame(df)
X = df.drop(columns = [106])
y = df[106]

sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X)

In [13]:
kf = KFold(n_splits=10, shuffle=False)

models = {
    "MLP": MLPClassifier(hidden_layer_sizes=(64, 32, 16, 8), activation='logistic', max_iter=10000, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=10),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

model_metrics = {name: {"Accuracy": [], "Error": [], "Precision": [], "Recall": [], "F1-Score": [], "Kappa": []} for name in models.keys()}

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1} :")
    
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        acc = accuracy_score(y_test, y_pred)
        error = 1 - acc  # Error rate
        prec = precision_score(y_test, y_pred, average="weighted", zero_division=0)
        rec = recall_score(y_test, y_pred, average="weighted")
        f1 = f1_score(y_test, y_pred, average="weighted")
        kappa = cohen_kappa_score(y_test, y_pred)

        model_metrics[name]["Accuracy"].append(acc)
        model_metrics[name]["Error"].append(error)
        model_metrics[name]["Precision"].append(prec)
        model_metrics[name]["Recall"].append(rec)
        model_metrics[name]["F1-Score"].append(f1)
        model_metrics[name]["Kappa"].append(kappa)

        print(f"{name}: Acc={acc:.4f}, Err={error:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}, Kappa={kappa:.4f}")

    print("-" * 50)

table_data = [
    [name, 
     f"{np.mean(metrics['Accuracy']):.4f}", 
     f"{np.mean(metrics['Error']):.4f}", 
     f"{np.mean(metrics['Precision']):.4f}", 
     f"{np.mean(metrics['Recall']):.4f}", 
     f"{np.mean(metrics['F1-Score']):.4f}", 
     f"{np.mean(metrics['Kappa']):.4f}"]
    for name, metrics in model_metrics.items()
]

print(tabulate(table_data, headers=["Model", "Avg Acc", "Avg Error", "Avg Prec", "Avg Recall", "Avg F1", "Avg Kappa"], tablefmt="grid"))

Fold 1 :
MLP: Acc=0.9991, Err=0.0009, Prec=0.9991, Rec=0.9991, F1=0.9991, Kappa=0.9988
Random Forest: Acc=0.9981, Err=0.0019, Prec=0.9981, Rec=0.9981, F1=0.9981, Kappa=0.9977
SVM: Acc=0.9992, Err=0.0008, Prec=0.9992, Rec=0.9992, F1=0.9992, Kappa=0.9990
KNN: Acc=0.9976, Err=0.0024, Prec=0.9976, Rec=0.9976, F1=0.9976, Kappa=0.9970
Logistic Regression: Acc=0.9984, Err=0.0016, Prec=0.9984, Rec=0.9984, F1=0.9984, Kappa=0.9980
Decision Tree: Acc=0.9948, Err=0.0052, Prec=0.9948, Rec=0.9948, F1=0.9948, Kappa=0.9935
--------------------------------------------------
Fold 2 :
MLP: Acc=0.9984, Err=0.0016, Prec=0.9984, Rec=0.9984, F1=0.9984, Kappa=0.9980
Random Forest: Acc=0.9988, Err=0.0012, Prec=0.9988, Rec=0.9988, F1=0.9988, Kappa=0.9985
SVM: Acc=0.9991, Err=0.0009, Prec=0.9991, Rec=0.9991, F1=0.9991, Kappa=0.9988
KNN: Acc=0.9972, Err=0.0028, Prec=0.9972, Rec=0.9972, F1=0.9972, Kappa=0.9965
Logistic Regression: Acc=0.9988, Err=0.0012, Prec=0.9988, Rec=0.9988, F1=0.9988, Kappa=0.9985
Decision Tr

In [14]:
from sklearn.metrics import confusion_matrix

labels = np.unique(y)
conf_matrix_sum = {name: np.zeros((len(labels), len(labels))) for name in models.keys()}

for fold, (train_idx, test_idx) in enumerate(kf.split(df)):
    print(f"Fold {fold + 1} :")
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        cm = confusion_matrix(y_test, y_pred, labels=labels)
        conf_matrix_sum[name] += cm
        print(f"{name}")

    print(f"Fold {fold + 1} Completed")
    print("-" * 50)

conf_matrix_total = {name: cm.astype(int) for name, cm in conf_matrix_sum.items()}

conf_matrix_data = []
for i, actual in enumerate(labels):
    for name, cm in conf_matrix_total.items():
        row = [actual, name]
        row.extend(cm[i])
        conf_matrix_data.append(row)

columns = ["Actual Class", "Algorithm"] + list(labels)
df_conf_matrix = pd.DataFrame(conf_matrix_data, columns=columns)

print(df_conf_matrix)

Fold 1 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 1 Completed
--------------------------------------------------
Fold 2 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 2 Completed
--------------------------------------------------
Fold 3 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 3 Completed
--------------------------------------------------
Fold 4 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 4 Completed
--------------------------------------------------
Fold 5 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 5 Completed
--------------------------------------------------
Fold 6 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 6 Completed
--------------------------------------------------
Fold 7 :
MLP
Random Forest
SVM
KNN
Logistic Regression
Decision Tree
Fold 7 Completed
--------------------------------------------------
Fold 8 :
MLP
Random Forest
SVM
KNN
Logist

In [15]:
print(tabulate(df_conf_matrix, headers='keys', tablefmt='grid'))

+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|    | Actual Class   | Algorithm           |   Arborio |   Basmati |   Ipsala |   Jasmine |   Karacadag |
+====+================+=====================+===========+===========+==========+===========+=============+
|  0 | Arborio        | MLP                 |     14965 |         0 |        0 |        24 |          11 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  1 | Arborio        | Random Forest       |     14970 |         0 |        0 |        23 |           7 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  2 | Arborio        | SVM                 |     14973 |         0 |        0 |        16 |          11 |
+----+----------------+---------------------+-----------+-----------+----------+-----------+-------------+
|  3 | Arborio        | KNN          